In [102]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,f1_score,accuracy_score,confusion_matrix
from sklearn.metrics import roc_curve,auc,roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import re
from collections import Counter

a=" bad"
kk='[{"feed": "positive", "text": "'+a+'" },{  "feed": "jkjkj",   "text": "bad" }]'
TweetDataSubset=pd.read_json(kk)
nltk.download("stopwords")
stop_words=set(stopwords.words("english"))
wordnet = WordNetLemmatizer()
def text_preproc(x):
    
    x=''.join([word for word in x.split(' ')if word not in stop_words])
    x=x.encode('ascii','ignore').decode()
    x=re.sub(r'https*\S+',' ',x)
    x=re.sub(r'@\S+',' ',x)
    x=re.sub(r'#\s+',' ',x)
    x=re.sub(r'\'\w+','',x)
    x=re.sub('[%s]'% re.escape(string.punctuation),'',x)
    x=re.sub(r'\w*\d+\w*','',x)
    x=re.sub(r'\s{2,}','',x)
    return x
TweetDataSubset['clean_text']=TweetDataSubset.text.apply(text_preproc)
x_train,x_test,y_train,y_test=train_test_split(TweetDataSubset["clean_text"],TweetDataSubset["feed"],test_size=0.1,shuffle=True)
tfidf_vectorizer=TfidfVectorizer(use_idf=True,max_features=100)
x_train_vectors_tfidf=tfidf_vectorizer.fit_transform(x_train)
x_test_vectors_tfidf=tfidf_vectorizer.transform(x_test)
lr_tfidf=MultinomialNB()
lr_tfidf.fit(x_train_vectors_tfidf,y_train)

y_predict=lr_tfidf.predict(x_test_vectors_tfidf)
y_prob=lr_tfidf.predict_proba(x_test_vectors_tfidf)[:]
Counter("".join(TweetDataSubset['clean_text']).split()).most_common(100)



Positive_pattern_1=r"thanks"
Positive_pattern_2=r"thank"
Positive_pattern_3=r"good"
Positive_pattern_4=r"well"
Positive_pattern_5=r"super"
Positive_pattern_6=r"awesome"
Positive_pattern_7=r"satisfied"
Positive_pattern_8=r"great"

Postive_Pattern_List=[Positive_pattern_1,Positive_pattern_2,Positive_pattern_3,Positive_pattern_4,Positive_pattern_5,Positive_pattern_6,Positive_pattern_7,Positive_pattern_8]

Positive_Complex_Pattern=re.compile('|'.join(['(%s)'% i for i in Postive_Pattern_List]),re.IGNORECASE)


Negative_pattern_1=r"cancelled"
Negative_pattern_2=r"delayed"
Negative_pattern_3=r"trying"
Negative_pattern_4=r"please"
Negative_pattern_5=r"wait"
Negative_pattern_6=r"worst"
Negative_pattern_7=r"bad"
Negative_pattern_8=r"tough"
Negative_Pattern_List=[Negative_pattern_1,Negative_pattern_2,Negative_pattern_3,Negative_pattern_4,Negative_pattern_5,Negative_pattern_6,Negative_pattern_7,Negative_pattern_8]

Negative_Complex_Pattern=re.compile('|'.join(['(%s)'% i for i in Negative_Pattern_List]),re.IGNORECASE)

TweetDataSubset["Negative_Sentiment_Flag"]=TweetDataSubset["clean_text"].apply(lambda x:1 if(len(re.findall(Negative_Complex_Pattern,x))>0) else 0)

TweetDataSubset["Positive_Sentiment_Flag"]=TweetDataSubset["clean_text"].apply(lambda x:1 if(len(re.findall(Positive_Complex_Pattern,x))>0) else 0)

TweetDataSubset

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KESAVAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,feed,text,clean_text,Negative_Sentiment_Flag,Positive_Sentiment_Flag
0,positive,bad,bad,1,0
1,jkjkj,bad,bad,1,0


In [103]:
res=TweetDataSubset.values.tolist()
res

[['positive', ' bad', 'bad', 1, 0], ['jkjkj', 'bad', 'bad', 1, 0]]

In [107]:
a=res[0][3]
b=res[0][4]
p2=b-a
if(p2 >0 ):
    print('postive')
else:
    tt=a+0.5
    print(tt)

1.5
